# 📱 Blocks Cloud Phone Launcher
**Instructions:**
1. Click the **Play** button below.
2. Wait for the phone to boot (approx. 2-3 minutes).
3. Click the **Localtunnel** link that appears at the end to see your phone!

In [ ]:
# 1. Install System Requirements
!apt-get update
!apt-get install -y docker.io nodejs npm adb

# 2. Start the Android Engine (Redroid)
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest

# 3. Clone and Build the Viewer directly from Source
!git clone https://github.com/NetrisTV/ws-scrcpy.git
%cd ws-scrcpy
!npm install
!npm run dist

# 4. Install Localtunnel so you can see the site publicly
!npm install -g localtunnel

# 5. Run everything and show the link
import subprocess
import threading

def run_viewer():
    subprocess.run(["npm", "start", "--", "--port", "7860"])

threading.Thread(target=run_viewer, daemon=True).start()

print("\n✅ Phone is booting! Click the link below to open it:")
!lt --port 7860